In [ ]:
import glob
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.utils import to_categorical
from keras.layers.convolutional import Conv1D, MaxPooling1D

In [ ]:
# datasetsフォルダ内にあるcsvファイルの一覧を取得
def get_data(dataset_dir:str, content:dict, t_range:int):
  files = {num: glob.glob(f"{dataset_dir}/{cont}/muscle*") for num, cont in content.items()}
  datas = [decode(file, num, t_range) for num, file in files.items()]
  return assemble_table(datas)

def decode(files:list, label:int, t_range:int):
  name = lambda x: int(re.search(r"\d+\.", x).group(0).strip("."))
  table = {name(f): pd.read_csv(f, header=None, dtype=np.float32)[1] for f in files}
  df = pd.DataFrame(table, index=np.arange(t_range)).T
  l = pd.Series(np.full(100, label), name=1000, index=df.index)
  return pd.concat([df,l], axis=1)

def assemble_table(data:list):
  return pd.concat(data, axis=0)

# history plot
def plot_history(fit):
    fig, (axL, axR) = plt.subplots(ncols=2, figsize=(10,4))
    # Plot the loss in the history
    axL.plot(fit.history['loss'],label="loss for training")
    axL.plot(fit.history['val_loss'],label="loss for validation")
    axL.set_title('model loss')
    axL.set_xlabel('epoch')
    axL.set_ylabel('loss')
    axL.legend(loc='upper right')

    axR.plot(fit.history['accuracy'],label="loss for training")
    axR.plot(fit.history['val_accuracy'],label="loss for validation")
    axR.set_title('model accuracy')
    axR.set_xlabel('epoch')
    axR.set_ylabel('accuracy')
    axR.legend(loc='upper right')
    
def preprocess(func, data):
  d = data.copy()
  d.loc[:,0:999] = d.loc[:,0:999].apply(func, axis=1)
  return d

In [ ]:
##### CONST NUMBERS SETTING #####
# Data settings
dataset_dir = "./Dataset/trial3"
content = {0:"inside", 1:"outside", 2:"fist"}
t_range = 1000

# Model settings
batch_size = 16
nb_epoch = 100
nb_class = 3

In [ ]:
##### Setup Train/Test Data #####
# Load Data
data = get_data(dataset_dir, content, t_range).sort_index()

# Pre Processing
# e.x.) 
pre1 = lambda x: pd.Series(fft(x))
# pre2 = lambda x: pd.Series(np.log(x))
data = preprocess(pre1, data)

# Prepare train, test data
ex_var = data.loc[:, 0:999].values
obj_var = data.loc[:, 1000].values

_x_train, _x_test, _y_train, _y_test = train_test_split(ex_var, obj_var, test_size=0.3, random_state=0, shuffle=True)

x_train, y_train = _x_train.reshape(-1, 1000, 1), to_categorical(_y_train)
x_test, y_test = _x_test.reshape(-1, 1000, 1), to_categorical(_y_test)

print(f"""
Dataset shape
x_train: {x_train.shape} |-> y_train: {y_train.shape},
x_test: {x_test.shape} |-> y_test: {y_test.shape}
""")


In [ ]:
##### Model #####
# Define model
model = Sequential()

model.add(Conv1D(filters=128, input_shape=(1000,1), kernel_size=2, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Activation('relu'))

model.add(Conv1D(filters=128, kernel_size=2, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Activation('relu'))

model.add(Conv1D(filters=128, kernel_size=2, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = "rmsprop",
              metrics = ['accuracy'])

model.summary()

In [ ]:
# Learning
if __name__=="__main__":
  es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
  fname = 'weights.{epoch:02d}-{loss:.2f}-{accuracy:.2f}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5'
  cp_cb = keras.callbacks.ModelCheckpoint(filepath="./checkout/"+fname, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

  learning_rates = np.linspace(0.03, 0.001, nb_epoch)
  lr_cb = keras.callbacks.LearningRateScheduler(lambda epoch: float(learning_rates[epoch]))

  history = model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, validation_data=(x_test, y_test), verbose=1, callbacks=[ cp_cb, lr_cb])
  score = model.evaluate(x_test, y_test, batch_size=batch_size)
  print(list(zip(model.metrics_names, score)))
  plot_history(history)

In [ ]:
n = 10
test = x_test[0:n], y_test[0:n]
result = model.predict(test[0])

for i in range(n):
  p = np.argmax(result[i])
  t = np.argmax(test[1][i])
  print(f"test{i}: predict...{content[p]} / ground truth...{content[t]}\n  test{i}:{'○' if p == t else '☓'}")